# Deliverable 1. Retrieve Weather Data.
---
1. Create a folder called `Weather_Database` to save all the files related with this deliverable.

2. Save the `Weather_Database_starter_code.ipynb` starter code to the `Weather_Database` folder and rename it as `Weather_Database.ipynb`.

3. Use the `np.random.uniform` function to generate a new set of 2,000 random latitudes and 2,000 longitudes.

4. Use the `citipy` module to get the nearest city for each latitude and longitude combination.

5. Import your OpenWeatherMap's API key and assemble the API call URL as a string variable. Recall to edit the `config.py` file to add your API key; also, it's critical to avoid publishing your API key on your GitHub repository.

6. Retrieve the following information from the API call:

    * Latitude and longitude

    * Maximum temperature

    * Percent humidity

    * Percent cloudiness

    * Wind speed

    * Weather description (for example, clouds, fog, light rain, clear sky)

7. Add the weather data to a new DataFrame.

8. Export the DataFrame as a CSV file, and save it as `WeatherPy_Database.csv` in the `Weather_Database` folder.
---

In [2]:
# Import initial libraries
import numpy as np
import pandas as pd
from citipy import citipy
import random

## Use the `np.random.uniform` function to generate a new set of 2,000 random latitudes and 2,000 longitudes.

In [3]:
# Create a set of random latitude and longitude combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)

# Use the zip function to create an iterator of tuples containing the latitude and longitude combinations
lat_lngs = zip(lats, lngs)
lat_lngs

In [4]:
# Add the latitudes and longitudes to a list
coordinates = list(lat_lngs)

In [5]:
for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

-1.4469933726853839 169.11601907438097
-56.267420499957744 -178.1165315873058
71.68115241104636 -150.5302237551508
72.8073527914016 -142.82703146684932
62.296190950234205 -86.18514707757355
-44.02637364878229 -1.1328252064083415
-29.46291456377108 -39.91547375869814
-66.34836783235747 -33.84162809052361
-22.174339222365205 124.84521275105516
30.155011225587174 -82.05081251164185
3.214530983290473 14.8792967039644
-18.457789945198073 -155.8687403119886
-88.6227188832366 143.72942105193522
-28.631212680166932 67.14478993855096
-46.430219083 -52.49408615132876
83.48672098406459 -153.09960362812646
72.11094185169509 134.30155602807486
57.63944846372459 54.03728283760634
-4.515731583377317 -102.56981451936872
-16.510486094678782 -100.86165066229013
23.624730550840056 -94.73855054079398
-53.0318410121969 118.12234093396313
51.159079770012454 -122.04298863578029
-13.057645125389655 -95.11782741198088
3.997571769235691 17.047057522271274
-72.67926855191212 -62.45257028646199
-50.13594981016792

## Use the `citipy` module to get the nearest city for each latitude and longitude combination.

In [6]:
# Create an empty list for holding the cities
cities = []
# Use a for loop to identify nearest city for each latitude and longitude combination using the citipy module
for coordinate in coordinates:
    # Use the citipy module to identify the nearest city to the current coordinate
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

736

## Import your OpenWeatherMap's API key and assemble the API call URL as a string variable. Recall to edit the `config.py` file to add your API key.

In [7]:
# Import the requests library
import requests

# Import the time library
import time

# Import the datetime module from the datetime library
from datetime import datetime

# Import the OpenWeatherMap's API key
from config import weather_api_key

In [8]:
# Assemble the OpenWeatherMap's API call
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key


## Retrieve the following information from the API call.

- The latitude and longitude
- The Max temperature
- The % humidity
- The % cloudiness
- The Wind speed
- The Weather description, i.e., cloudy, fog, light rain, clear sky, etc. 

In [9]:
# Create an empty list to hold weather data for each city
city_data = []

# Print a message to indicate that the data retrieval starts
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters and set them to 1
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data for each city
for i, city in enumerate(cities):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create an endpoint URL for each city
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log the url, record, and set numbers
    print(f"Processing Record {record_count} of Set {set_count} | {city}")

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        city_weather = requests.get(city_url).json()
        # Parse out the latitude, longitude, max temp, humidity, cloudiness, wind, country, and weather description
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]  
        city_description = city_weather["weather"][0]["description"]

        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
            
        # Append the city information into the city_data list
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Description": city_description
                          })
    
    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass

# Indicate that the data retrieval is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | tabiauea
City not found. Skipping...
Processing Record 2 of Set 1 | vaini
Processing Record 3 of Set 1 | barrow
Processing Record 4 of Set 1 | aklavik
Processing Record 5 of Set 1 | attawapiskat
City not found. Skipping...
Processing Record 6 of Set 1 | cape town
Processing Record 7 of Set 1 | arraial do cabo
Processing Record 8 of Set 1 | ushuaia
Processing Record 9 of Set 1 | broome
Processing Record 10 of Set 1 | lakeside
Processing Record 11 of Set 1 | yokadouma
Processing Record 12 of Set 1 | vaitape
Processing Record 13 of Set 1 | hobart
Processing Record 14 of Set 1 | bambous virieux
Processing Record 15 of Set 1 | mar del plata
Processing Record 16 of Set 1 | nizhneyansk
City not found. Skipping...
Processing Record 17 of Set 1 | sharkan
Processing Record 18 of Set 1 | puerto ayora
Processing Record 19 of Set 1 | nuevo progreso
Processing Record 20 of Set 1 | albany
Processing Record 21 o

In [10]:
# Print the length of the city_data list to verify how many cities you have
len(city_data)

682

## Add the weather data to a new DataFrame.

In [11]:
# Use the city_data list to create a new pandas DataFrame.
city_data_df = pd.DataFrame(city_data)

In [12]:
# Display sample data
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Description
0,Vaini,-21.2000,-175.2000,73.56,94,100,10.36,TO,moderate rain
1,Barrow,71.2906,-156.7887,6.82,78,100,16.11,US,overcast clouds
2,Aklavik,68.2191,-135.0107,-0.26,98,100,0.85,CA,light snow
3,Cape Town,-33.9258,18.4232,72.37,79,9,7.00,ZA,clear sky
4,Arraial Do Cabo,-22.9661,-42.0278,75.18,86,100,3.69,BR,overcast clouds
5,Ushuaia,-54.8000,-68.3000,42.46,87,0,5.75,AR,light rain
6,Broome,42.2506,-75.8330,17.02,88,0,9.22,US,clear sky
7,Lakeside,30.1300,-81.7682,64.45,85,75,10.36,US,broken clouds
8,Yokadouma,3.5167,15.0500,75.45,58,100,2.68,CM,overcast clouds
9,Vaitape,-16.5167,-151.7500,78.93,73,100,18.70,PF,overcast clouds


In [13]:
# Display the DataFrame's column names using the columns Pandas function
city_data_df.columns

Index(['City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Description'],
      dtype='object')

In [14]:
# Create a list to reorder the column names as follows:
# "City", "Country", "Lat", "Lng", "Max Temp", "Humidity",  "Cloudiness", "Wind Speed",  "Current Description"
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Description"]

# Recreate the DataFrame by using the new column order
clean_city_data_df = city_data_df[new_column_order] 

# Display sample data
clean_city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,Vaini,TO,-21.2000,-175.2000,73.56,94,100,10.36,moderate rain
1,Barrow,US,71.2906,-156.7887,6.82,78,100,16.11,overcast clouds
2,Aklavik,CA,68.2191,-135.0107,-0.26,98,100,0.85,light snow
3,Cape Town,ZA,-33.9258,18.4232,72.37,79,9,7.00,clear sky
4,Arraial Do Cabo,BR,-22.9661,-42.0278,75.18,86,100,3.69,overcast clouds
5,Ushuaia,AR,-54.8000,-68.3000,42.46,87,0,5.75,light rain
6,Broome,US,42.2506,-75.8330,17.02,88,0,9.22,clear sky
7,Lakeside,US,30.1300,-81.7682,64.45,85,75,10.36,broken clouds
8,Yokadouma,CM,3.5167,15.0500,75.45,58,100,2.68,overcast clouds
9,Vaitape,PF,-16.5167,-151.7500,78.93,73,100,18.70,overcast clouds


In [15]:
# Display the data types of each column by using the dtypes Pandas function
clean_city_data_df.dtypes

City            object
Country         object
Lat            float64
Lng            float64
Max Temp       float64
Humidity         int64
Cloudiness       int64
Wind Speed     float64
Description     object
dtype: object

## Export the DataFrame as a CSV file, and save it as `WeatherPy_Database.csv` in the `Weather_Database` folder.

In [19]:
# Set the output file name
output_data_file = "WeatherPy_Database.csv"

# Export the city_data DataFrame into a CSV file
clean_city_data_df.to_csv(output_data_file, index_label="City_ID")